In [ ]:
from pathlib import Path
import pandas as pd
from typing import Dict
def export_to_excel(
    directory: Path,
    file_name: str,
    summary_details_df : pd.DataFrame,
    table_comparison: Dict[str, Dict[str, pd.DataFrame],
    metadata_comparison: Dict[str, Dict],
    logger: logging.Logger,)->bool:
    summary_count = {}
    try:
        file_path = Path(directory, file_name)
        for table_name, dataframes_dict in table_comparison.items():
            summary_count[table_name] = _export_dict_to_excel(file_path,table_name, dataframes_dict, logger)
            summary_count_df = _create_summary_count_df(summary_count, logger)
            status = _construct_summary_sheet(file_path,summary_details_df, summary_count_df, logger)
        return True

    except Exception as e:
        logger.error(f"Error occured")
        return False

def _export_dict_to_excel(file_path: Path, table_name: str, data_dict: Dict[str, pd.DataFrame], logger: Logging.Logger)->Dict[str, int]
:
    summary_count = {}
    with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
        writer.book.create_sheet(title=table_name)
        sheet = writer.sheets[table_name]
        start_row = 0
        for label, df in data_dict.items():
            # Write the label
            sheet.cell(row=start_row + 1, column=1, value=label)
            # Write the DataFrame below the label
            df.to_excel(writer, sheet_name=table_name, startrow=start_row + 1, index=True)
            # Update the start_row for the next label and DataFrame
            start_row += len(df) + 3  # +3 for the label and two blank rows
            if label in ["new_columns", "deleted_columns"]:
                count = df.shape[1]
            else:
                count = df.shape[0]
            summary_count[label] = count
        return summary_count

def create_summary_count_df(data_dict: Dict[str, Dict[str, int]]) -> pd.DataFrame:
    # Define static columns with their values
    static_columns = {"Identical": 0, "New Table": 0}
    
    # Get dynamic columns from the first dictionary in data_dict
    dynamic_columns = list(next(iter(data_dict.values())).keys())
    
    # Define all columns
    columns = ["Table"] + list(static_columns.keys()) + dynamic_columns + ["comments"]
    
    # Create rows using list comprehension
    rows = [
        {
            "Table": table_name,
            **{col: count_dict.get(col, 0) for col in dynamic_columns},
            **static_columns,
            "comments": "none"
        }
        for table_name, count_dict in data_dict.items()
    ]
    
    # Create the DataFrame from the list of rows
    df_summary = pd.DataFrame(rows, columns=columns)
    
    return df_summary
            
def _construct_summary_sheet(file_path : Path,summary_details_df: pd.DataFrame, summary_count_df: pd.DataFrame, logger: logging.Logger)->
bool:
    sheet_name = "summary"
    with pd.ExcelWriter(file_path, engine = "openpyxl") as writer:
        summary_details_df.to_excel(writer, sheet_name = sheet_name, index = False, startrow = 0 , header = False)
        start_row = len(summary_details_df) + 3
        summary_count_df.to_excel(writer, sheet_name = sheet_name, index = False, startrow = start_row)
    return True
    

In [ ]:
#chat gpt version

In [ ]:
from pathlib import Path
import pandas as pd
import logging
from typing import Dict

def export_to_excel(
    directory: Path,
    file_name: str,
    summary_details_df: pd.DataFrame,
    table_comparison: Dict[str, Dict[str, pd.DataFrame]],
    metadata_comparison: Dict[str, Dict],
    logger: logging.Logger) -> bool:
    summary_count = {}
    try:
        file_path = directory / file_name
        
        # Ensure the directory exists
        directory.mkdir(parents=True, exist_ok=True)
        
        # Create the Excel file and write to it using xlsxwriter
        with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
            # Add worksheets and write DataFrames
            for table_name, dataframes_dict in table_comparison.items():
                summary_count[table_name] = _export_dict_to_excel(writer, table_name, dataframes_dict, logger)
            
            summary_count_df = create_summary_count_df(summary_count)
            status = _construct_summary_sheet(writer, summary_details_df, summary_count_df, logger)
        
        return status

    except Exception as e:
        logger.error(f"Error occurred: {e}")
        return False

def _export_dict_to_excel(writer: pd.ExcelWriter, table_name: str, data_dict: Dict[str, pd.DataFrame], logger: logging.Logger) -> Dict[str, int]:
    summary_count = {}
    try:
        # Get the workbook and add a worksheet
        workbook = writer.book
        worksheet = workbook.add_worksheet(table_name)
        
        start_row = 0
        for label, df in data_dict.items():
            # Write the label
            worksheet.write_string(start_row, 0, label)
            
            # Write the DataFrame below the label
            df.to_excel(writer, sheet_name=table_name, startrow=start_row + 1, index=True, header=True)
            
            # Update the start_row for the next label and DataFrame
            start_row += len(df) + 3  # +3 for the label and two blank rows
            
            # Count the number of columns or rows based on the label
            if label in ["new_columns", "deleted_columns"]:
                count = df.shape[1]
            else:
                count = df.shape[0]
            
            summary_count[label] = count

    except Exception as e:
        logger.error(f"Error occurred while exporting dict to Excel: {e}")

    return summary_count

def create_summary_count_df(data_dict: Dict[str, Dict[str, int]]) -> pd.DataFrame:
    # Define static columns with their values
    static_columns = {"Identical": 0, "New Table": 0}
    
    # Get dynamic columns from the first dictionary in data_dict
    dynamic_columns = list(next(iter(data_dict.values())).keys())
    
    # Define all columns
    columns = ["Table"] + list(static_columns.keys()) + dynamic_columns + ["comments"]
    
    # Create rows using list comprehension
    rows = [
        {
            "Table": table_name,
            **{col: count_dict.get(col, 0) for col in dynamic_columns},
            **static_columns,
            "comments": "none"
        }
        for table_name, count_dict in data_dict.items()
    ]
    
    # Create the DataFrame from the list of rows
    df_summary = pd.DataFrame(rows, columns=columns)
    
    return df_summary

def _construct_summary_sheet(writer: pd.ExcelWriter, summary_details_df: pd.DataFrame, summary_count_df: pd.DataFrame, logger: logging.Logger) -> bool:
    sheet_name = "summary"
    try:
        # Get the workbook and add a worksheet
        workbook = writer.book
        worksheet = workbook.add_worksheet(sheet_name)
        
        # Write summary_details_df to the summary sheet
        summary_details_df.to_excel(writer, sheet_name=sheet_name, index=False, startrow=0, header=False)
        
        start_row = len(summary_details_df) + 3
        # Write summary_count_df to the summary sheet
        summary_count_df.to_excel(writer, sheet_name=sheet_name, index=False, startrow=start_row)
        
        return True

    except Exception as e:
        logger.error(f"Error occurred while constructing summary sheet: {e}")
        return False
